In [ ]:
import pymysql
import pandas as pd

# Configuración BD
DB_HOST = "192.168.245.33"
DB_USER = "compensaciones_rrhh"
DB_PASSWORD = "_Cramercomp2025_"
DB_NAME = "rrhh_app"

pais = "CHILE" # "PERU" o "CHILE"

In [ ]:
def conectar_y_extraer_datos():
    """
    Conecta a la base de datos y extrae las columnas específicas en un DataFrame
    """
    try:
        print("🚀 Conectando a MySQL...")
        
        # Establecer conexión
        conexion = pymysql.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME,
            charset='utf8mb4'
        )

        print("Conexión exitosa.")

        # if "PERU" in pais:
        #     query_personas = """
        #         SELECT
        #             empleado.full_name AS nombre,
        #             empleado.document_number AS dni,
        #             empleado.name_role AS cargo,
        #             empleado.base_wage AS sueldo_base,
        #             jefe.full_name AS nombre_jefe,
        #             jefe.name_role AS cargo_jefe,
        #             -- Calcula la edad en años
        #             TIMESTAMPDIFF(YEAR, empleado.birthday, CURDATE()) AS edad,
        #             -- Calcula el tiempo en la empresa en años
        #             TIMESTAMPDIFF(YEAR, empleado.active_since, CURDATE()) AS años_en_la_empresa,
        #             -- Calcula el tiempo en meses
        #             TIMESTAMPDIFF(MONTH, empleado.active_since, CURDATE()) AS meses_en_la_empresa
        #         FROM
        #             rrhh_app.employees_peru AS empleado
        #         LEFT JOIN
        #             rrhh_app.employees_peru AS jefe ON empleado.dni_boss = jefe.document_number
        #         WHERE
        #             empleado.status = "activo";
        #         """
        #     query_cargos = """
        #         SELECT
        #             name_role AS cargo,
        #             MIN(base_wage) AS sueldo_base_minimo,
        #             MAX(base_wage) AS sueldo_base_maximo
        #         FROM
        #             rrhh_app.employees_peru
        #         WHERE
        #             status = "activo"
        #         GROUP BY
        #             name_role
        #         ORDER BY
        #             name_role;
        #     """
        if "CHILE" in pais:
            # query_personas = """
            #     SELECT
            #         empleado.full_name AS nombre,
            #         empleado.rut,
            #         empleado.name_role AS cargo,
            #         empleado.base_wage AS sueldo_base,
            #         jefe.full_name AS nombre_jefe,
            #         jefe.name_role AS cargo_jefe,
            #         -- Calcula la edad en años
            #         TIMESTAMPDIFF(YEAR, empleado.birthday, CURDATE()) AS edad,
            #         -- Calcula el tiempo en la empresa en años
            #         TIMESTAMPDIFF(YEAR, empleado.active_since, CURDATE()) AS años_en_la_empresa,
            #         -- Calcula el tiempo en meses
            #         TIMESTAMPDIFF(MONTH, empleado.active_since, CURDATE()) AS meses_en_la_empresa
            #     FROM
            #         rrhh_app.employees AS empleado
            #     LEFT JOIN
            #         rrhh_app.employees AS jefe ON empleado.rut_boss = jefe.rut
            #     WHERE
            #         empleado.status = "activo";
            #     """
            query_cargos = """
                SELECT
                    t1.name_role AS cargo,
                    t2.second_level_name AS nombre_division,
                    t2.name AS nombre_area,
                    MIN(t1.base_wage) AS sueldo_base_minimo,
                    MAX(t1.base_wage) AS sueldo_base_maximo,
                    COUNT(*) AS cantidad_empleados -- CAMBIO 1: Se usa COUNT(*) o COUNT(t1.name_role)

                FROM
                    rrhh_app.employees t1
                INNER JOIN
                    rrhh_app.areas t2 ON t1.cost_center = t2.cost_center

                WHERE
                    t1.status = 'activo'

                GROUP BY
                    t1.name_role,
                    t2.second_level_name,
                    t2.name  -- CAMBIO 2: Se eliminó la coma extra aquí

                ORDER BY
                    t1.name_role,
                    t2.second_level_name,
                    t2.name;
            """
        
        # Crear DataFrame directamente desde la query
        #df_personas = pd.read_sql(query_personas, conexion)
        df_cargos = pd.read_sql(query_cargos, conexion)

        # print(f"📋 Columnas disponibles en df_personas: {list(df_personas.columns)}")
        print(f"📋 Columnas disponibles en df_cargos: {list(df_cargos.columns)}")
        # Cerrar conexión
        conexion.close()
        print("✅ Conexión cerrada correctamente.")

        return df_cargos #df_personas, 
        
    except Exception as e:
        print(f"❌ Error al conectar o extraer datos: {e}")
        return None

In [18]:

def export_query_to_excel(df_cargos): #df_personas
    # Exportar DataFrame a Excel
    if "PERU" in pais:
        ruta_archivo = "db_jobs_peru.xlsx"
    elif "CHILE" in pais:
        ruta_archivo = "db_jobs_chile.xlsx"
    with pd.ExcelWriter(ruta_archivo, engine="openpyxl") as writer:
        # df_personas.to_excel(writer, sheet_name="Personas", index=False)   
        df_cargos.to_excel(writer, sheet_name="Cargos", index=False)  
    print(f"Archivo Excel creado en: {ruta_archivo}")


In [19]:
if __name__ == "__main__":
    df_cargos = conectar_y_extraer_datos() #df_personas,
    if  df_cargos is not None: #df_personas is not None and
        export_query_to_excel( df_cargos) #df_personas,


🚀 Conectando a MySQL...
Conexión exitosa.
📋 Columnas disponibles en df_cargos: ['cargo', 'nombre_division', 'nombre_area', 'sueldo_base_minimo', 'sueldo_base_maximo', 'cantidad_empleados']
✅ Conexión cerrada correctamente.


C:\Users\gpavez\AppData\Local\Temp\ipykernel_13900\1882214857.py:107: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cargos = pd.read_sql(query_cargos, conexion)


Archivo Excel creado en: db_jobs_chile.xlsx
